In [43]:
import math
import numpy as np

# --- helper 함수들 ---
#회전 행렬 만들기
#x축으로 roll, y축으로 pitch, z축으로 yaw
def rpy_to_mat(roll, pitch, yaw): 
    
    Rx = np.array([[1, 0, 0],
                   [0, math.cos(roll), -math.sin(roll)],
                   [0, math.sin(roll),  math.cos(roll)]])
    Ry = np.array([[ math.cos(pitch), 0, math.sin(pitch)],
                   [0, 1, 0],
                   [-math.sin(pitch), 0, math.cos(pitch)]])
    Rz = np.array([[math.cos(yaw), -math.sin(yaw), 0],
                   [math.sin(yaw),  math.cos(yaw), 0],
                   [0, 0, 1]])
    return Rz @ Ry @ Rx


def axis_angle_to_mat(axis, angle):
    axis = np.array(axis) / np.linalg.norm(axis) #단위 벡터로 만들기
    x,y,z = axis #축을 x,y,z로 나누기
    c, s = math.cos(angle), math.sin(angle) #삼각함수 각도 구하기
    C = 1-c #1-cos 저장
    # Rodrigues 회전 공식 적용
    return np.array([
      [x*x*C + c,   x*y*C - z*s, x*z*C + y*s],
      [y*x*C + z*s, y*y*C + c,   y*z*C - x*s],
      [z*x*C - y*s, z*y*C + x*s, z*z*C + c  ]
    ])

In [84]:
import json
from urchin import URDF

ROOT_PATH = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001'
# 1. URDF와 data.json 파일 경로
URDF_PATH   = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/g1_inspire_hand_description/g1_29dof_rev_1_0_with_inspire_hand_FTP.urdf'    # URDF가 담긴 .txt 파일
DATA_PATH   = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/data.json'   # data.json 파일


finger_map = {'thumb':5, 'thumb2':4, 'index':3, 'middle':2, 'ring':1, 'pinky':0}

LEFTRIGHT = 'right'
WHICH_FINGER = 'middle'
FRAME = 0

names = [LEFTRIGHT, 'shoulder', 'elbow', 'wrist', 'pitch', 'roll', 'yaw']   #얘는 list로 비교하면서 바꿨고 (아래 for i, joint in enumerate(joints) 부분에서)
handmap = {
                f'k{LEFTRIGHT.capitalize()}HandThumbRotation' : f'{LEFTRIGHT}_thumb_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandThumbBend' : f'{LEFTRIGHT}_thumb_3_joint',
                f'k{LEFTRIGHT.capitalize()}HandIndex' : f'{LEFTRIGHT}_index_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandMiddle' : f'{LEFTRIGHT}_middle_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandRing' : f'{LEFTRIGHT}_ring_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandPinky' : f'{LEFTRIGHT}_little_1_joint',
            }

# 2. URDF 로드
robot = URDF.load(URDF_PATH, lazy_load_meshes=True)

# 3. data.json 로드
with open(DATA_PATH, 'r') as f:
    data = json.load(f)


joints  = data['info']['joint_names'][f'{LEFTRIGHT}_arm']  # 7개 joint names list       여기부터

for i, joint in enumerate(joints):
    tmp_name = ""

    joint = joint[1:]
    for name in names:
        find_res = joint.lower().find(name)
        if find_res == -1:
            continue
        else:
            tmp_name = tmp_name + joint[:find_res+len(name)] + '_'
            joint = joint[find_res+len(name):]
    
    joints[i] = (tmp_name[:-1] + '_joint').lower()

# 4. idx=0의 left_arm 관절각(qpos)과 joint 이름 목록 가져오기
qpos    = data['data'][FRAME]['states'][f'{LEFTRIGHT}_arm']['qpos']                         #여기까지는 arm을 뽑는부분

hand_qpos  = data['data'][FRAME]['states'][f'{LEFTRIGHT}_hand']['qpos']                 #여기부터
hand_joints = data['info']['joint_names'][f'{LEFTRIGHT}_hand']
hand_joints = [handmap[x] for x in hand_joints]                                       #여기까지는 hand joints 정리해두는부분 이중에 한개만 뽑아서 넣을거임

# 5. joint name → angle 매핑
q_dict = {name: angle for name, angle in zip(joints, qpos)}

q_dict[hand_joints[finger_map[WHICH_FINGER]]] = hand_qpos[finger_map[WHICH_FINGER]]       #hand q_dict에 추가

# 6. forward kinematics 계산  
# ee_link = f'{LEFTRIGHT}_wrist_yaw_link'     # wrist뽑을때 uncomment, hand 뽑을때 comment
ee_link = hand_joints[finger_map[WHICH_FINGER]][:-6]                    # hand 뽑을때 uncomment, wrist 뽑을때 comment
print(ee_link)
# link_fk()는 각 링크별로 월드 기준 4×4 변환행렬을 반환
fk = robot.link_fk(q_dict)
# for link_obj, T in fk.items():
#     print(link_obj.name)
# T_ee_world = fk[ee_link]   # (4,4) 행렬
ee_link_obj  = robot.link_map[ee_link]    # 이름→Link 객체 얻기
T_ee_world   = fk[ee_link_obj]                 # 올바르게 인덱싱!

# 7. 위치(translation) 추출
pos_ee_world = T_ee_world[:3, 3]
print('Left arm end-effector position in world frame:', pos_ee_world)


right_middle_1
Left arm end-effector position in world frame: [ 0.46655618 -0.13741504  0.05557407]


In [ ]:
# cam = robot['d435I_joint']
T_world_cam = np.eye(4)
T_world_cam[:3,:3] = rpy_to_mat(0, 0.8307767239493009, 0)
T_world_cam[:3, 3] = [0.0576235, 0.01753, 0.42987]   # :contentReference[oaicite:0]{index=0}


# 5) camera→left_wrist 변환
T_cam_ee = np.linalg.inv(T_world_cam) @ T_ee_world

# 6) translation 성분이 바로 “카메라 기준” 손끝 좌표
pos_cam_ee = T_cam_ee[:3, 3]
print("Left wrist position in D435 camera frame:", pos_cam_ee)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
Left wrist position in D435 camera frame: [ 0.55214511 -0.15494504  0.04958988]


0프레임 :[ 0.46655618 -0.13741504  0.05557407]

0프레임 :   [0.22070608 0.18346118 0.01026804] 이게 맨처음 정지위치쯤 이때 위치가 이렇다는건 일단 허리쪽이 0,0,0인건 확실히 맞긴한듯?

카메라기준 : [ 0.41982428  0.16593118 -0.1625094 ] 

roll일때: [ 0.16308258 -0.19796951 -0.40547137]

-----

50프레임 : [0.29628327 0.1957955  0.08448675] 위로만 올렸을때

카메라기준 : [ 0.41597896  0.1782655  -0.05665317]

roll일때 : [ 0.23865977 -0.13484525 -0.36453386]

------

143프레임 : [0.35340309 0.20754136 0.17797557] 이게 0보다 살짝왼쪽, 꽤 위, 꽤 앞

카메라기준 : [0.38545768 0.19001136 0.048567  ]

roll일때 : [ 0.29577959 -0.05788768 -0.31016792]

------

285프레임 : [0.3584677  0.18067249 0.18189677] 이게 143이랑 높이는 비슷하고..가 아니라 다 비슷한게 맞음

카메라기준 : [0.38597712 0.16314249 0.05495107]

------

405프레임 : [0.33813684 0.20274944 0.1394626 ] 약간 인사하는거같은 손위치. x는 비슷한거같고(살~짝 오른쪽?), y는 좀더 뒤로왔고, z는 거의 비슷한듯

카메라기준 : [0.40360371 0.18521944 0.01132417]

roll일때 : [ 0.28051334 -0.08955899 -0.33259869]

In [ ]:
import cv2
import numpy as np

# 1) Intrinsics
width, height = 848, 480
fx = 426.7954
fy = 426.7954
cx = width  / 2  # 424
cy = height / 2  # 320

# 2) pos_cam_ee: (X, Y, Z)  
#    X: 오른쪽(+), Y: 카메라 앞쪽(+), Z: 위쪽(+)
print(pos_cam_ee)
Xc =  pos_cam_ee[0]      
Yc =  pos_cam_ee[1]      
Zc =   pos_cam_ee[2]     

# 2) 투영
u = fx * (Yc / Xc) + cx
v = fy * (Zc / Xc) + cy

# 소수점→정수 픽셀 좌표
print(u,v)
pt = (int(round(848-u)), int(round(480-v)))
print(pt)
# 4) 이미지 읽고 표시  
img = cv2.imread("/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000596_color_0.jpg")
print(FRAME)
if img is None:
    raise RuntimeError("Unable to load image.")
cv2.circle(img, pt, 8, (255,0,0), thickness=-1)  # 빨간 점
cv2.putText(img, f"{pt}", (pt[0]+10, pt[1]),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

# 5) 결과 저장 (Jupyter에서 imshow 대신)
cv2.imwrite(f"projected_pointindex2{FRAME}.png", img)
print("Saved to projected_point.png")


520.5732963707565 177.50040585950345
(327, 302)
596
Saved to projected_point.png
